In [87]:
import pandas as pd


In [88]:
l = []
i=0
with open("/content/train.txt","r") as f:
  while(True):
    data = f.readline()
    l.append(data)
    i=i+1
    if(i==10000):
      break



In [89]:
b = []
for i in l:
  b.append(i.split(";",))

In [90]:
data = {"text":[],
        "emotion":[]}

In [91]:
for i in b:
  data["text"].append(i[0])
  data["emotion"].append(i[1][0:-1])


In [92]:
df = pd.DataFrame(data)

In [93]:
df

,text,emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger
...,...,...
9995,i have a desk job and sit on my ass all day lo...,fear
9996,i feel like such a confused person lately sigh,fear
9997,i feel ive been beaten down by the words of me...,sadness
9998,i fully understand the feeling of being beaten...,sadness


In [94]:
import spacy
nlp = spacy.load("en_core_web_sm")

def preprocess(text):
  l=[]
  doc = nlp(text)
  for i in doc:
    if(i.is_stop  or i.is_punct):
      continue
    l.append(i.lemma_)
  return " ".join(l)


In [95]:
df["text"] = df["text"].apply(preprocess)

In [96]:
df

,text,emotion
0,not feel humiliate,sadness
1,feel hopeless damned hopeful care awake,sadness
2,m grab minute post feel greedy wrong,anger
3,feel nostalgic fireplace know property,love
4,feel grouchy,anger
...,...,...
9995,desk job sit ass day long feel paranoid m acti...,fear
9996,feel like confused person lately sigh,fear
9997,feel ve beat word man ground not sleep beneath...,sadness
9998,fully understand feeling beat wound bereft,sadness


In [97]:
num_categories = set(df['emotion'])
df['emotion_num'] = df.emotion.map({
    'anger':5, 'surprise':2, 'sadness':3, 'fear':4, 'joy':1, 'love':0
})

In [98]:
df

,text,emotion,emotion_num
0,not feel humiliate,sadness,3
1,feel hopeless damned hopeful care awake,sadness,3
2,m grab minute post feel greedy wrong,anger,5
3,feel nostalgic fireplace know property,love,0
4,feel grouchy,anger,5
...,...,...,...
9995,desk job sit ass day long feel paranoid m acti...,fear,4
9996,feel like confused person lately sigh,fear,4
9997,feel ve beat word man ground not sleep beneath...,sadness,3
9998,fully understand feeling beat wound bereft,sadness,3


In [99]:
from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test = train_test_split(df['text'],df["emotion_num"],test_size=0.1)

In [139]:
from sklearn.feature_extraction.text import TfidfVectorizer

v = TfidfVectorizer(ngram_range=(1,5))
train_mat = v.fit_transform(X_train)
v.vocabulary_

{'get': 74376,
 'amazing': 3986,
 'news': 122110,
 'tracy': 175035,
 'today': 173763,
 'final': 68560,
 'cover': 27205,
 'chapter': 19550,
 'wonder': 189841,
 'feel': 46445,
 'rushed': 147531,
 'not': 122950,
 'time': 172059,
 'get amazing': 74398,
 'amazing news': 4067,
 'news tracy': 122158,
 'tracy today': 175036,
 'today final': 173857,
 'final cover': 68563,
 'cover chapter': 27210,
 'chapter wonder': 19562,
 'wonder feel': 189890,
 'feel rushed': 61815,
 'rushed not': 147532,
 'not time': 124884,
 'get amazing news': 74399,
 'amazing news tracy': 4068,
 'news tracy today': 122159,
 'tracy today final': 175037,
 'today final cover': 173858,
 'final cover chapter': 68564,
 'cover chapter wonder': 27211,
 'chapter wonder feel': 19563,
 'wonder feel rushed': 189916,
 'feel rushed not': 61816,
 'rushed not time': 147533,
 'get amazing news tracy': 74400,
 'amazing news tracy today': 4069,
 'news tracy today final': 122160,
 'tracy today final cover': 175038,
 'today final cover chapte

In [140]:
train_mat.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [141]:
test_mat = v.transform(X_test)

In [142]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(train_mat,Y_train)

MultinomialNB()

In [143]:
pred = model.predict(test_mat)

In [144]:

from sklearn.metrics import classification_report

print(classification_report(Y_test,pred))

              precision    recall  f1-score   support

           0       1.00      0.02      0.04        89
           1       0.55      0.99      0.71       334
           2       0.00      0.00      0.00        42
           3       0.74      0.88      0.81       287
           4       0.91      0.19      0.32       104
           5       0.97      0.24      0.39       144

    accuracy                           0.64      1000
   macro avg       0.70      0.39      0.38      1000
weighted avg       0.72      0.64      0.56      1000



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


###RandomForest

In [145]:
from sklearn.ensemble import RandomForestClassifier

model2 = RandomForestClassifier()
model2.fit(train_mat,Y_train)


RandomForestClassifier()

In [146]:
pred2 = model2.predict(test_mat)

In [147]:
from sklearn.metrics import classification_report,precision_recall_curve

print(classification_report(Y_test,pred2))

              precision    recall  f1-score   support

           0       0.85      0.64      0.73        89
           1       0.85      0.95      0.90       334
           2       0.86      0.74      0.79        42
           3       0.87      0.92      0.89       287
           4       0.89      0.81      0.85       104
           5       0.92      0.83      0.87       144

    accuracy                           0.87      1000
   macro avg       0.87      0.81      0.84      1000
weighted avg       0.87      0.87      0.87      1000



In [148]:
t = v.transform(["I’m so mad right now. I can’t talk to you!"])
model2.predict(t)

array([5])